### 投開票所轉換教學
<p>
此Python腳本示範如何將中選會投開票所的公開資訊轉換成選舉模組需要使用的對照表，在執行該檔案的腳本之前，請先利用election_hisotry中的
<a href="https://github.com/readr-data/election_history/blob/master/liCode_voter.Rmd">liCode_voter.Rmd</a>
腳本將網路上的投開票所資訊爬蟲到該資料夾底下，並設定cec_tbox_filepath為該檔案路徑。完成後再執行下面的函式產生對照表。
</p>

### 安裝並引入相依套件

In [ ]:
!pip install --upgrade pip
!pip install -r requirements.txt

In [ ]:
import json
import os
import copy
import pandas as pd
from collections import defaultdict

### 系統操作函式

In [ ]:
def save_file(destination_file, data):
    dirname = os.path.dirname(destination_file)
    if len(dirname)>0 and not os.path.exists(dirname):
        os.makedirs(dirname)
    with open(destination_file, 'w', encoding='utf-8') as json_file:
        json.dump(data, json_file, ensure_ascii=False, indent=4)
        print(f'Save data to {destination_file} successed.')

### 產生mapping表

In [ ]:
def gen_tbox_mapping(filepath: str):
    result = {}
    tbox_info = pd.read_csv(filepath)
    # tbox_info.columns = ['city', 'tboxNo', 'tboxName', 'address']
    tbox_info.columns = ['tboxNo', 'village', 'countyCode', 'townCode']
    tbox_info['village'] = tbox_info['village'].str.replace('\n','')
    tbox_info['countyCode'] = tbox_info['countyCode'].astype(str).apply(lambda x: x.zfill(5))
    tbox_info['townCode'] = tbox_info['townCode'].astype(str).apply(lambda x: x.zfill(3))
    tbox_info['tboxNo'] = tbox_info['tboxNo'].astype(str)

    for i in range(len(tbox_info)):
        data = tbox_info.iloc[i]
        countyCode = data['countyCode']
        townCode   = data['townCode']
        tboxNo     = data['tboxNo']
        village    = data['village']

        subCounty = result.setdefault(countyCode, {})
        subTown = subCounty.setdefault(townCode, {})
        subTbox = subTown.setdefault(tboxNo, [])
        subTbox.append(village)
    return result

In [ ]:
cec_tbox_filepath = os.path.join('.', 'cec-data', '20240113_tbox.csv') ### TODO: 需要使用爬蟲重新產生[time]_tbox.csv檔案
mapping_tbox = gen_tbox_mapping(cec_tbox_filepath)
mapping_tbox

在產生tbox的對照表後，請將其存檔並手動放入選舉模板中的mapping資料夾底下